In [ ]:
import pandas as pd
import re  
from google.colab import drive
import os
import fasttext
import numpy as np
import nltk
import itertools
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df=pd.read_csv('./drive/MyDrive/Comp 642 data/tweets_preprocessed.csv', lineterminator='\n')

Only use English tweets, which are identified by fasttext librarys.

In [ ]:
model = fasttext.load_model('./drive/MyDrive/Comp 642 data/lid.176.bin')
def detect_lang(text):
    try:
        language = model.predict(text)
        return language
    except:
        return 'unknown'

In [ ]:
cleaned_dataset = df.copy()
cleaned_dataset['language'] = cleaned_dataset['text'].apply(detect_lang)

In [ ]:
cleaned_dataset_eng = cleaned_dataset[cleaned_dataset['language'].apply(lambda x: x[0] == ('__label__en',))]

In [ ]:
cleaned_dataset_eng = cleaned_dataset_eng[cleaned_dataset_eng['language'].apply(lambda x: float(x[1][0]) > 0.5)]

In [ ]:
cleaned_dataset_eng = cleaned_dataset_eng[cleaned_dataset['text'] != '']

<ipython-input-115-8ef8fcbc428c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_dataset_eng = cleaned_dataset_eng[cleaned_dataset['text'] != '']


In [ ]:
cleaned_dataset_eng.head()

,created_at,text,language
7,2022-11-02 07:19:51+00:00,RT @darylginn: did it hurt? when they called y...,"((__label__en,), [0.9719023704528809])"
12,2022-11-02 07:19:51+00:00,RT @VoteWithARMY: I vote #YetToCome as #TheMus...,"((__label__en,), [0.8037695288658142])"
14,2022-11-02 07:19:51+00:00,RT @SexyNatG: #retweet if you want to come rub...,"((__label__en,), [0.7123409509658813])"
25,2022-11-02 07:19:51+00:00,"RT @thekayanova: I’m not sure how, when, or wh...","((__label__en,), [0.8766927123069763])"
28,2022-11-02 07:19:51+00:00,having to complete nilou's story quest made me...,"((__label__en,), [0.8141809701919556])"


In [ ]:
cleaned_dataset_eng.shape

(804807, 3)

In [ ]:
# Write the results to a CSV file
cleaned_dataset_eng.to_csv("./drive/MyDrive/Comp 642 data/tweets_english.csv",  mode='w', index=False)

# Clean the dataset

In [ ]:
# Define regular expressions
emoji_regex = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
not_emoji_regex = re.compile("[^[\U0001F300-\U0001F6FF]]")
emoticon_regex = re.compile("(?::|;|=)(?:-)?(?:\\)|D|P)")  # From https://stackoverflow.com/questions/5862490/how-to-match-emoticons-with-regular-expressions
not_emoticon_regex = re.compile("^(?::|;|=)(?:-)?(?:\\)|D|P)")
url_regex = re.compile("(https?://)?([\\w\\-])+\\.{1}([a-zA-Z]{2,63})([/\\w-]*)*/?\\??([^#\n\r]*)?#?([^\n\r]*)")  # Simpler => (http[^ ]*)|(www\.[^ ]*)
mention_regex = re.compile("(@[\\w_-]+)")
hashtag_regex = re.compile("(#[\\w_-]+)")

In [ ]:
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace(mention_regex, " ")
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace(url_regex, " ")
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace(hashtag_regex, " ")
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace(r"\s+", " ")
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace("RT : ", " ") # remove RT
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace('^[\s]+|[\s]+$', "") # remove starting and trailing space

# Build emoticons dictionary (might be not the optimal one, but it should cover the majority of the most used emoticons)
emots = ["".join(emot) for emot in itertools.product([':', ';', ':-', ';-'], [')', '(', ']', '[', 'D', 'o', 'O', 'P', 'p'])]

# Helper function for escaping special characters
def escape_regex(r):
    return re.sub(r'([()\[\]])', r'\\\1', r)

# Emoticon regular expression
regex1 = f"({'|'.join([escape_regex(emot) for emot in emots])})"

# Removing punctuation, but keeping the emoticons and ' (https://stackoverflow.com/questions/23447261/remove-punctuation-but-keeping-emoticons)
# We just match (emoticon)|punctuation (note that emoticons are in a capturing group) and then replace the matches with the result of capturing group 1 
# (so if it's an emoticon, we have replacement=this emoticon, if it's a punctuation char, we have replacement=nothing)
cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].apply(lambda x: re.sub(f"({regex1}|[^\\\P{{P}}'@#]+)", r"\1 ", x))

<ipython-input-120-8c9115881a24>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace(r"\s+", " ")
<ipython-input-120-8c9115881a24>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_dataset_eng['text'] = cleaned_dataset_eng['text'].str.replace('^[\s]+|[\s]+$', "") # remove starting and trailing space


In [ ]:
cleaned_dataset_eng = cleaned_dataset_eng[cleaned_dataset_eng['text'] != '']
cleaned_dataset_eng = cleaned_dataset_eng[cleaned_dataset_eng['text'].notna()]

In [ ]:
cleaned_dataset_eng.head(10)

,created_at,text,language
7,2022-11-02 07:19:51+00:00,did it hurt? when they called your finished de...,"((__label__en,), [0.9719023704528809])"
12,2022-11-02 07:19:51+00:00,I vote as of the year at the 2022 People’s Ch...,"((__label__en,), [0.8037695288658142])"
14,2022-11-02 07:19:51+00:00,if you want to come rub oil on me 🧴,"((__label__en,), [0.7123409509658813])"
25,2022-11-02 07:19:51+00:00,"I’m not sure how, when, or why, but I have a f...","((__label__en,), [0.8766927123069763])"
28,2022-11-02 07:19:51+00:00,having to complete nilou 's story quest made m...,"((__label__en,), [0.8141809701919556])"
34,2022-11-02 07:19:51+00:00,It took almost 10 years but BTS finally have s...,"((__label__en,), [0.9826110601425171])"
38,2022-11-02 07:19:51+00:00,cuz i don 't get the whole meeting lol maybe o...,"((__label__en,), [0.8963832855224609])"
42,2022-11-02 07:19:51+00:00,Remember to wear that smile everyday 😁,"((__label__en,), [0.7248052358627319])"
43,2022-11-02 07:19:51+00:00,When you’ve been warmongering as long as Puti...,"((__label__en,), [0.7731037139892578])"
67,2022-11-02 07:19:52+00:00,"SHAME ON KEYSTONE BANK and it’s Ceo, OLANIRAN ...","((__label__en,), [0.5829444527626038])"


In [ ]:
cleaned_dataset_eng.shape

(765762, 3)

In [ ]:
# Write the results to a CSV file
cleaned_dataset_eng.to_csv("./drive/MyDrive/Comp 642 data/cleaned_tweets_english.csv", mode='w', index=False)

# Identify sentiment for each tweet

In [ ]:
sia= SentimentIntensityAnalyzer()

In [ ]:
cleaned_dataset_eng['scores']=cleaned_dataset_eng['text'].apply(lambda body: sia.polarity_scores(body))

In [ ]:
# https://github.com/cjhutto/vaderSentiment
def calculate_sentiment(scores):
    score = scores["compound"]
    if score > 0.05:
        return 'positive'
    elif score < -0.05:
        return 'negative'
    else:
        return 'neutral'

# apply function to each row of the dataframe
cleaned_dataset_eng['sentiment'] = cleaned_dataset_eng['scores'].apply(calculate_sentiment)

In [ ]:
cleaned_dataset_eng.head()

,created_at,text,language,scores,sentiment
7,2022-11-02 07:19:51+00:00,did it hurt? when they called your finished de...,"((__label__en,), [0.9719023704528809])","{'neg': 0.224, 'neu': 0.592, 'pos': 0.184, 'co...",negative
12,2022-11-02 07:19:51+00:00,I vote as of the year at the 2022 People’s Ch...,"((__label__en,), [0.8037695288658142])","{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'comp...",positive
14,2022-11-02 07:19:51+00:00,if you want to come rub oil on me 🧴,"((__label__en,), [0.7123409509658813])","{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compou...",positive
25,2022-11-02 07:19:51+00:00,"I’m not sure how, when, or why, but I have a f...","((__label__en,), [0.8766927123069763])","{'neg': 0.05, 'neu': 0.71, 'pos': 0.24, 'compo...",positive
28,2022-11-02 07:19:51+00:00,having to complete nilou 's story quest made m...,"((__label__en,), [0.8141809701919556])","{'neg': 0.0, 'neu': 0.888, 'pos': 0.112, 'comp...",positive
...,...,...,...,...,...
4194716,2022-11-02 07:19:49+00:00,So you might be afraid when we say we want to ...,"((__label__en,), [0.9412142038345337])","{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...",positive
4194718,2022-11-02 07:19:49+00:00,This man found a unicorn of a woman who is a b...,"((__label__en,), [0.9331048727035522])","{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'comp...",positive
4194719,2022-11-02 07:19:49+00:00,“Sometimes your joy is the source of your smil...,"((__label__en,), [0.7717400789260864])","{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'comp...",positive
4194733,2022-11-02 07:19:50+00:00,"when I put “???” In a conversation, this is ex...","((__label__en,), [0.8959476947784424])","{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp...",positive


In [ ]:
cleaned_dataset_eng_sentiment = cleaned_dataset_eng[['text', 'sentiment']]

In [ ]:
cleaned_dataset_eng_sentiment.head()

,text,sentiment
7,did it hurt? when they called your finished de...,negative
12,I vote as of the year at the 2022 People’s Ch...,positive
14,if you want to come rub oil on me 🧴,positive
25,"I’m not sure how, when, or why, but I have a f...",positive
28,having to complete nilou 's story quest made m...,positive


In [ ]:
cleaned_dataset_eng_sentiment.shape

(765762, 2)

In [ ]:
cleaned_dataset_eng_sentiment.to_csv("./drive/MyDrive/Comp 642 data/cleaned_dataset_eng_sentiment.csv", mode='w', index=False)

# dataset without emojis

In [ ]:
cleaned_dataset_eng_sentiment_no = cleaned_dataset_eng_sentiment.copy()

In [ ]:
cleaned_dataset_eng_sentiment_no['text'] = cleaned_dataset_eng_sentiment_no['text'].str.replace(emoji_regex, " ")
cleaned_dataset_eng_sentiment_no['text'] = cleaned_dataset_eng_sentiment_no['text'].str.replace(emoticon_regex, " ")
cleaned_dataset_eng_sentiment_no['text'] = cleaned_dataset_eng_sentiment_no['text'].str.replace('^[\s]+|[\s]+$', "") # remove starting and trailing space
cleaned_dataset_eng_sentiment_no['text'] = cleaned_dataset_eng_sentiment_no['text'].str.replace(r"\s+", " ")

<ipython-input-172-70b73f9f23d2>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_dataset_eng_sentiment_no['text'] = cleaned_dataset_eng_sentiment_no['text'].str.replace('^[\s]+|[\s]+$', "") # remove starting and trailing space
<ipython-input-172-70b73f9f23d2>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cleaned_dataset_eng_sentiment_no['text'] = cleaned_dataset_eng_sentiment_no['text'].str.replace(r"\s+", " ")


In [ ]:
cleaned_dataset_eng_sentiment_no.head(10)

,text,sentiment
7,did it hurt? when they called your finished de...,negative
12,I vote as of the year at the 2022 People’s Cho...,positive
14,if you want to come rub oil on me,positive
25,"I’m not sure how, when, or why, but I have a f...",positive
28,having to complete nilou 's story quest made m...,positive
34,It took almost 10 years but BTS finally have s...,neutral
38,cuz i don 't get the whole meeting lol maybe o...,positive
42,Remember to wear that smile everyday,positive
43,When you’ve been warmongering as long as Putin...,negative
67,"SHAME ON KEYSTONE BANK and it’s Ceo, OLANIRAN ...",negative


In [ ]:
cleaned_dataset_eng_sentiment_no.shape

(765762, 2)

In [ ]:
cleaned_dataset_eng_sentiment_no.to_csv("./drive/MyDrive/Comp 642 data/cleaned_dataset_eng_sentiment_no_emoji_emoticon.csv", mode='w', index=False)